In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping
from PIL import Image
import numpy as np

new_size = (64, 64)
resized_images = []
images=[]
for img in images:
    try:
        # Ensure the array is of dtype uint8 and has 3 dimensions (height, width, channels)
        img_array = np.asarray(img, dtype=np.uint8)
        if img_array.ndim == 2:
            # Add a channel dimension for grayscale images
            img_array = np.expand_dims(img_array, axis=-1)
        elif img_array.ndim != 3 or img_array.shape[-1] != 3:
            raise ValueError("Invalid image array shape")

        # Resize the image
        resized_img_array = np.array(Image.fromarray(img_array).resize(new_size))
        resized_images.append(resized_img_array)
    except Exception as e:
        print(f"Error processing image: {e}")

# Continue with the rest of your code...



# Assuming target size for InceptionV3 is (299, 299)
target_size = (299, 299)

def load_and_preprocess_image(file_path, target_size=target_size):
    image = tf.keras.preprocessing.image.load_img(file_path, target_size=target_size)
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = tf.keras.applications.inception_v3.preprocess_input(image_array)
    return image_array

def create_model(target_size):
    base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
    
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def train_model(model, train_generator, val_generator, epochs=10):
    lr_schedule = LearningRateScheduler(lambda epoch: 1e-3 if epoch < 5 else (1e-4 if 5 <= epoch < 10 else 1e-5))
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=[lr_schedule, early_stopping])

def evaluate_model(model, test_generator):
    results = model.evaluate(test_generator)
    print("Test Loss:", results[0])
    print("Test Accuracy:", results[1])

# Paths for clean and dirt images
clean_folder = "C:/Users/likhitha/Downloads/data/CleanSyntheticData"
dirt_folder = "C:/Users/likhitha/Downloads/data/DirtSyntheticData"
# Get a list of file paths for clean and dirt images
clean_files = [os.path.join(clean_folder, file) for file in os.listdir(clean_folder)]
dirt_files = [os.path.join(dirt_folder, file) for file in os.listdir(dirt_folder)]

# Create a DataFrame with file paths and labels (0 for clean, 1 for dirt)
data = {'file_path': clean_files + dirt_files, 'label': [0] * len(clean_files) + [1] * len(dirt_files)}
df = pd.DataFrame(data)

# Optionally, you can shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# Display the DataFrame information
print("DataFrame Information:")
print(df.info())

# Load and preprocess images
df['image'] = df['file_path'].apply(load_and_preprocess_image)

# Standardize pixel values
scaler = StandardScaler()

# Resize and load images into NumPy array
images = np.array([np.array(image) for image in df['image']])

# Standardize pixel values directly
images_standardized = scaler.fit_transform(images.reshape(images.shape[0], -1))

# Ensure that the standardized array has a consistent shape
images_standardized = np.vstack(images_standardized).reshape(images.shape)

print(len(df['file_path']))
print(len(df['label']))
print(len(images_standardized))

# Output the DataFrame with standardized pixel values
df_final = pd.DataFrame({'file_path': df['file_path'], 'label': df['label'].astype(str), 'image': images_standardized.tolist()})
print("\nDataFrame with Standardized Pixel Values:")
print(df_final.head())

# Get a list of file paths for clean and dirt images
clean_files = [os.path.join(clean_folder, file) for file in df_final['file_path']]
dirt_files = [os.path.join(dirt_folder, file) for file in df_final['file_path']]

# Create a DataFrame with file paths and labels
# Pad all lists in the 'image' column with zeros
max_length = max(len(image) for image in df_final['image'])
df_final['image'] = df_final['image'].apply(lambda x: x + [0] * (max_length - len(x)))


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_final['file_path'], df_final['label'], test_size=0.2, random_state=42)

# Create an InceptionV3 base model
model = create_model(target_size)

# Create data generators with data augmentation for training
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

# No data augmentation for validation/testing
test_datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32
# Define the validation generator
val_generator = test_datagen.flow_from_dataframe(dataframe=df_final, directory=None, x_col='file_path', y_col='label', target_size=target_size, batch_size=batch_size, class_mode='binary', shuffle=False)

# Train the model
batch_size = 32
train_generator = train_datagen.flow_from_dataframe(dataframe=df_final, directory=None, x_col='file_path', y_col='label', target_size=target_size, batch_size=batch_size, class_mode='binary', shuffle=True)

# Evaluate the model on the test set
test_generator = test_datagen.flow_from_dataframe(dataframe=df_final,directory=None,x_col='file_path',y_col='label',target_size=target_size,batch_size=batch_size,class_mode='binary',shuffle=False)
# Train and evaluate the model
train_model(model, train_generator, val_generator, epochs=10)
evaluate_model(model, test_generator)

# Load the trained model from the HDF5 file
model = tf.keras.models.load_model('model4.h5')



DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810 entries, 0 to 809
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_path  810 non-null    object
 1   label      810 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 12.8+ KB
None


In [ ]:
# Save the entire model including optimizer state
model.save('my_model.h5')

# Load the model
loaded_model = tf.keras.models.load_model('my_model.h5')
--


model = tf.keras.models.load_model('model4.h5')



In [ ]:
import tensorflow as tf
# Load the trained model from the HDF5 file
model = tf.keras.models.load_model('model4.h5')
def load_and_preprocess_new_image(file_path, target_size=target_size):
    img = tf.keras.preprocessing.image.load_img(file_path, target_size=target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.inception_v3.preprocess_input(img_array)
    return img_array
new_images_folder = "C:/Users/likhitha/OneDrive/Desktop/WEKA User defined datasets/SIH/Test"
new_image_files = [os.path.join(new_images_folder, file) for file in os.listdir(new_images_folder)]
new_image_predictions = []

for file_path in new_image_files:
    # Load and preprocess the new image
    new_image = load_and_preprocess_new_image(file_path)

    # Make predictions using the trained model
    prediction_result = model.predict(np.expand_dims(new_image, axis=0))

    # Interpret the prediction result
    if prediction_result[0][0] > 0.5:
        prediction_label = "Dirt Buildup"
    else:
        prediction_label = "Clean"

    new_image_predictions.append({
        'file_path': file_path,
        'prediction_label': prediction_label,
        'prediction_prob': prediction_result[0][0]
    })

# Print predictions after processing all images
for prediction in new_image_predictions:
    print(f"File: {prediction['file_path']}")
    print(f"Prediction: {prediction['prediction_label']}")
    print(f"Prediction Probability: {prediction['prediction_prob']:.4f}\n")